# Create Quickbase spreadsheet

Quickbase is a database tool which is being used as part of the Collections Information inventory process.
We want to load all the TEI records into Quickbase, without somebody having to copy/paste all the values manually.

This is a [Jupyter notebook] that was written collaboratively to create 

[Jupyter notebook]: https://jupyter.org

---

Housekeeping notes:

* Real tools only!
* I’ll add this notebook to the repo later, so you can see what we’ve done
* Please ask questions!
* We’ll take a comfort break at some point

Rough agenda:

* Find all the TEI files in the repo
* Extract the fields we want
* Write those fields to a CSV

In [2]:
print("Hello world!")
print("Hello Cassidy, Tacey, Adrian, Yoshika!")

Hello world!
Hello Cassidy, Tacey, Adrian, Yoshika!


# Find all the TEI files in the repo

```
for element in collection:
    do something with element
```

In [8]:
import os

# . = current directory

def find_tei_files(repo_path):
    for dirpath, dirnames, filenames in os.walk(repo_path):
        for f in filenames:
            path = os.path.join(dirpath, f)

            if not path.endswith('.xml'):
                continue

            if path.startswith('./Templates/'):
                continue

            if path.startswith('./docs/'):
                continue

            if path.startswith('./Arabic/Fihrist/'):
                continue

            yield path


for path in find_tei_files('.'):
    print(path)

./WMS_Arabic_65.xml
./Malay/Wellcome_MS_Malay_10.xml
./Malay/Wellcome_MS_Malay_8.xml
./Malay/Wellcome_MS_Malay_9.xml
./Malay/Wellcome_MS_Malay_7.xml
./Malay/Wellcome_MS_Malay_6.xml
./Malay/Wellcome_MS_Malay_4.xml
./Malay/Wellcome_MS_Malay_5.xml
./Malay/Wellcome_MS_Malay_1.xml
./Malay/Wellcome_MS_Malay_2.xml
./Malay/Wellcome_MS_Malay_3.xml
./Karshuni/Karshuni_3.xml
./Karshuni/Karshuni_2.xml
./Karshuni/Karshuni_1.xml
./Indic/Indic_Alpha_2075.xml
./Indic/B_20_m.xml
./Indic/E.11.v.xml
./Indic/E.11.a.xml
./Indic/Indic_Alpha_2101.xml
./Indic/Indic_Alpha_2129.xml
./Indic/Indic_Alpha_2128.xml
./Indic/Indic_Alpha_2100.xml
./Indic/Indic_Alpha_2114.xml
./Indic/Indic_Alpha_91.xml
./Indic/E.11.w.xml
./Indic/Indic_Alpha_124.xml
./Indic/B_20_l.xml
./Indic/Indic_Alpha_2074.xml
./Indic/Indic_Alpha_2076.xml
./Indic/B_20_n.xml
./Indic/Indic_Alpha_2089.xml
./Indic/Indic_Alpha_1999.xml
./Indic/E.11.u.xml
./Indic/E.11.b.xml
./Indic/P.81.I.xml
./Indic/Indic_Alpha_2102.xml
./Indic/Indic_Alpha_2103.xml
./Indic

# Extract the fields we want

In [64]:
from xml.etree import ElementTree as ET

def extract_tei_metadata_fields(tei_file_paths):
    for path in tei_file_paths:
        print(path)

        root = ET.parse(path).getroot()

        msContents = root.find('.//{http://www.tei-c.org/ns/1.0}msContents')

        # msContents/summary

        summary_tag = msContents.find('.//{http://www.tei-c.org/ns/1.0}summary')

        if summary_tag is not None:
            summary = cleanup_text(''.join(summary_tag.itertext()))
        else:
            summary = ''

        # <idno type="msID">

        idno = root.find('.//{http://www.tei-c.org/ns/1.0}idno[@type="msID"]').text

        # <altIdentifier>

        identifiers_sierra = []
        identifiers_accession_number = []
        identifiers_other = []

        for identifier in root.findall('.//{http://www.tei-c.org/ns/1.0}altIdentifier'):
            identifier_contents = identifier.find('.//{http://www.tei-c.org/ns/1.0}idno').text
            
            if not identifier_contents:
                continue

            if identifier.attrib.get('type') == 'Sierra':
                identifiers_sierra.append(identifier_contents)

            elif identifier.attrib.get('type') == 'accession':
                identifiers_accession_number.append(identifier_contents)

            else:
                identifiers_other.append(identifier_contents)

        # <textLang>

        languages = []

        for lang in msContents.findall('.//{http://www.tei-c.org/ns/1.0}textLang'):
            if not lang.text:
                continue
            
            if lang.text in languages:
                continue

            languages.append(lang.text)

        yield {
            'Title/s or Brief Description (245)': summary,
            'Main Identifier': idno,
            # MS1; MS2; MS3
            'Alternative Identifiers (separate with semicolons)': '; '.join(identifiers_other),
            'Accession Number': '; '.join(identifiers_accession_number),
            'System No.': '; '.join(identifiers_sierra),
            # Arabic; Hebrew; Hindi
            'Language': '; '.join(languages),
        }

In [27]:
'; '.join(['a', 'b', 'c'])

'a; b; c'

In [ ]:
field = 'fish'

if field == '':
    print("it's empty!")
else:
    print("it's not empty!")

# Write those fields to a CSV

In [65]:
import csv

with open('quickbase_spreadsheet.csv', 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=[
            'Title/s or Brief Description (245)',
            'Main Identifier',
            'Alternative Identifiers (separate with semicolons)',
            'Accession Number',
            'System No.',
            'Language',
        ]
    )
    
    writer.writeheader()
    
    tei_file_paths = find_tei_files('.')

    for row in extract_tei_metadata_fields(tei_file_paths):
        writer.writerow(row)

./WMS_Arabic_65.xml
./Malay/Wellcome_MS_Malay_10.xml
./Malay/Wellcome_MS_Malay_8.xml
./Malay/Wellcome_MS_Malay_9.xml
./Malay/Wellcome_MS_Malay_7.xml
./Malay/Wellcome_MS_Malay_6.xml
./Malay/Wellcome_MS_Malay_4.xml
./Malay/Wellcome_MS_Malay_5.xml
./Malay/Wellcome_MS_Malay_1.xml
./Malay/Wellcome_MS_Malay_2.xml
./Malay/Wellcome_MS_Malay_3.xml
./Karshuni/Karshuni_3.xml
./Karshuni/Karshuni_2.xml
./Karshuni/Karshuni_1.xml
./Indic/Indic_Alpha_2075.xml
./Indic/B_20_m.xml
./Indic/E.11.v.xml
./Indic/E.11.a.xml
./Indic/Indic_Alpha_2101.xml
./Indic/Indic_Alpha_2129.xml
./Indic/Indic_Alpha_2128.xml
./Indic/Indic_Alpha_2100.xml
./Indic/Indic_Alpha_2114.xml
./Indic/Indic_Alpha_91.xml
./Indic/E.11.w.xml
./Indic/Indic_Alpha_124.xml
./Indic/B_20_l.xml
./Indic/Indic_Alpha_2074.xml
./Indic/Indic_Alpha_2076.xml
./Indic/B_20_n.xml
./Indic/Indic_Alpha_2089.xml
./Indic/Indic_Alpha_1999.xml
./Indic/E.11.u.xml
./Indic/E.11.b.xml
./Indic/P.81.I.xml
./Indic/Indic_Alpha_2102.xml
./Indic/Indic_Alpha_2103.xml
./Indic

./Arabic/MS_Arabic_861.xml
./Arabic/WMS_Arabic_483.xml
./Arabic/WMS_Arabic_497.xml
./Arabic/WMS_Arabic_468.xml
./Arabic/WMS_Arabic_440.xml
./Arabic/WMS_Arabic_454.xml
./Arabic/WMS_Arabic_478.xml
./Arabic/WMS_Arabic_450.xml
./Arabic/WMS_Arabic_444.xml
./Arabic/WMS_Arabic_487.xml
./Arabic/MS_Arabic_865.xml
./Arabic/MS_Arabic_871.xml
./Arabic/WMS_Arabic_646.xml
./Arabic/WMS_Arabic_691.xml
./Arabic/WMS_Arabic_726.xml
./Arabic/WMS_Arabic_22.xml
./Arabic/WMS_Arabic_256.xml
./Arabic/WMS_Arabic_530.xml
./Arabic/WMS_Arabic_281.xml
./Arabic/WMS_Arabic_257.xml
./Arabic/WMS_Arabic_531.xml
./Arabic/WMS_Arabic_733.xml
./Arabic/WMS_Arabic_848.xml
./Arabic/WMS_Arabic_690.xml
./Arabic/WMS_Arabic_121.xml
./Arabic/WMS_Arabic_653.xml
./Arabic/MS_Arabic_858.xml
./Arabic/MS_Arabic_870.xml
./Arabic/MS_Arabic_864.xml
./Arabic/WMS_Arabic_486.xml
./Arabic/WMS_Arabic_445.xml
./Arabic/WMS_Arabic_451.xml
./Arabic/WMS_Arabic_479.xml
./Arabic/WMS_Arabic_447.xml
./Arabic/WMS_Arabic_453.xml
./Arabic/WMS_Arabic_484.xml

./Ethiopian/Ethiopian_7.xml
./Ethiopian/Ethiopian_14.xml
./Ethiopian/Ethiopian_15.xml
./Ethiopian/Ethiopian_6.xml
./Ethiopian/Ethiopian_4.xml
./Ethiopian/Ethiopian_17.xml
./Ethiopian/Ethiopian_16.xml
./Ethiopian/Ethiopian_5.xml
./Ethiopian/Ethiopian_1.xml
./Ethiopian/Ethiopian_12.xml
./Ethiopian/Ethiopian_13.xml
./Ethiopian/Ethiopian_10.xml
./Ethiopian/Ethiopian_3.xml
./Ethiopian/Ethiopian_21.xml
./Ethiopian/Ethiopian_20.xml
./Ethiopian/Ethiopian_22.xml
./Ethiopian/Ethiopian_23.xml
./Ethiopian/Ethiopian_8.xml
./Ethiopian/Ethiopian_27.xml
./Ethiopian/Ethiopian_26.xml
./Ethiopian/Ethiopian_9.xml
./Ethiopian/Ethiopian_18.xml
./Ethiopian/Ethiopian_24.xml
./Ethiopian/Ethiopian_25.xml
./Ethiopian/Ethiopian_19.xml


In [53]:
root = ET.parse('Hebrew/Hebrew_B_41.xml').getroot()

msContents = root.find('.//{http://www.tei-c.org/ns/1.0}msContents')

print(msContents)

summary_tag = msContents.find('.//{http://www.tei-c.org/ns/1.0}summary')

if summary_tag:
    print("it's not empty!")
    summary = ''.join(summary_tag.itertext())
else:
    print("it's empty!")
    summary = ''

print(summary_tag)
print(''.join(summary_tag.itertext()))

<Element '{http://www.tei-c.org/ns/1.0}msContents' at 0x111f97040>
it's empty!
<Element '{http://www.tei-c.org/ns/1.0}summary' at 0x111f97680>
Biblical fragment


In [63]:
def cleanup_text(s):
    lines = s.splitlines()
    
    trimmed_lines = [line.strip() for line in lines]
    
    return ' '.join(trimmed_lines)

demons = '''Charms and demonology. Though the term "demonology" is used in the manuscript itself, the creatures being described are known in Malay as "hantu", 
                            which differs from the Western conception of demons in that the term is commonly used to describe spirits and mythological creatures in general, both
                            malicious and not. Though some may also be called demons or syaitan in the Islamic sense, not all hantu are necessarily demonic.'''

print(cleanup_text(demons))

Charms and demonology. Though the term "demonology" is used in the manuscript itself, the creatures being described are known in Malay as "hantu", which differs from the Western conception of demons in that the term is commonly used to describe spirits and mythological creatures in general, both malicious and not. Though some may also be called demons or syaitan in the Islamic sense, not all hantu are necessarily demonic.
